In [1]:
import numpy as np
import warnings
from joblib import Parallel, delayed
from numba import jit

In [2]:
def user_pipeline(data):
    return np.mean(data, axis=1, keepdims=True)

@jit(nopython=True)
def loo_(data):
    n_features, n_samples = data.shape
    loo_means = np.zeros((n_features, n_samples))
    loo_stds = np.zeros((n_features, n_samples))

    for i in range(n_features):
        v = data[i]
        s, s2 = np.sum(v), np.sum(v**2)
        means = (s - v) / (n_samples - 1)
        #vars_ = ((s2 - v**2) - (n_samples - 1) * means**2) / (n_samples - 2)
        #loo_means[i], loo_stds[i] = means, np.sqrt(vars_)
        loo_means[i] = means
        
    return loo_means, loo_stds

@jit(nopython=True)
def gen_noise(chol_factor, sensitivity, max_attempts=10000):
    dim = sensitivity.shape[0]
    for _ in range(max_attempts):
        z = np.random.standard_normal(dim)
        noise = chol_factor @ z
        if np.all(noise >= sensitivity):
            return noise, True
    return noise, False

def dp(data, original_output, epsilon=1.0):
    loo_means, loo_stds = loo_(data)
    sensitivity = np.max(np.abs(loo_means - original_output), axis=1)

    loo_scales = np.std(loo_means, axis=1) / epsilon
    cov = np.cov(data)
    scale_factors = 2 * loo_scales / np.sqrt(np.diag(cov))
    scale_matrix = np.diag(scale_factors)

    noise_cov = scale_matrix @ cov @ scale_matrix
    if np.min(np.linalg.eigvals(noise_cov)) <= 1e-10:
        noise_cov += np.eye(data.shape[0]) * 1e-8

    try:
        chol = np.linalg.cholesky(noise_cov)
        noise, success = gen_noise(chol, sensitivity)
        if not success:
            raise ValueError("Correlated noise generation failed")
    except (np.linalg.LinAlgError, ValueError):
        warnings.warn("Failed to generate correlated noise, using independent noise")
        noise = np.array([np.random.normal(0, 2 * s) for s in loo_scales])
        for i in range(len(noise)):
            while abs(noise[i]) < sensitivity[i]:
                noise[i] = np.random.normal(0, 2 * loo_scales[i])

    noisy_outputs = original_output.flatten() + noise
    return noisy_outputs, sensitivity

In [100]:
def main(data):
    original_output = user_pipeline(data)
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always")
        noisy_outputs, sensitivities = dp(data, original_output)
        failed = any("Failed to generate correlated noise" in str(msg.message) for msg in w)

    result = {f'dp_t{i+1}': noisy_outputs[i] for i in range(len(noisy_outputs))}
    result.update({f'sensitivity_t{i+1}': sensitivities[i] for i in range(len(sensitivities))})
    return result, int(failed)


if __name__ == "__main__":
    #cov_matrix = np.array([[1.0, 0.5], [0.5, 1.0]])
    #data = np.random.multivariate_normal([0, 0], cov_matrix, 20000).T
    
    cov_matrix = np.array([[1.0, 0.5, 0.3],
                           [0.5, 1.0, 0.2],
                           [0.3, 0.2, 1.0]])
    data = np.random.multivariate_normal([0, 0, 0], cov_matrix, 20000).T
    import time
    s = time.time()
    results = Parallel(n_jobs=20, verbose=0, batch_size=1)(
        delayed(main)(data) for _ in range(2)
    )
    total_time = time.time() - s
    print(f"\nTotal time taken size [3,20000]: {time.strftime('%H:%M:%S', time.gmtime(total_time))}")
    size =  [400,50000]
    s = time.time()
    data = np.random.normal(loc=0.0, scale=1.0, size=size)
    results = Parallel(n_jobs=20, verbose=0, batch_size=1)(
            delayed(main)(data) for _ in range(2)
        )
    total_time = time.time() - s
    print(f"\nTotal time taken size [400,50000]: {time.strftime('%H:%M:%S', time.gmtime(total_time))}")

    


Total time taken size [3,20000]: 00:00:10

Total time taken size [400,50000]: 00:00:03


In [18]:
import warnings
warnings.filterwarnings("ignore")
def main(data):
    loo_data = [np.delete(data, i, axis=0) for i in range(data.shape[0])]
    test = np.array(loo_data)
    original_output = user_pipeline(data)
    noisy_outputs, sensitivities = dp(data, original_output)

    return noisy_outputs, sensitivities 


if __name__ == "__main__":
    
    data = np.random.randn(40,2000000)
    import time
    s = time.time()
    results = Parallel(n_jobs=20, verbose=0, batch_size=1)(
        delayed(main)(data) for _ in range(1)
    )
    total_time = time.time() - s
    print(f"\nTotal time taken: {time.strftime('%H:%M:%S', time.gmtime(total_time))}")
  


Total time taken: 00:00:01


/tmp/ipykernel_33769/2571096367.py:49: UserWarning: Failed to generate correlated noise, using independent noise
/tmp/ipykernel_33769/2571096367.py:49: UserWarning: Failed to generate correlated noise, using independent noise


In [21]:
noisy_outputs, sensitivities  = results

In [25]:
print(noisy_outputs)
print()
print(np.array(noisy_outputs).shape)

(array([ 0.07210078,  0.08122679,  0.01357883, -0.12030439]), array([0.01377031, 0.01381223, 0.0127531 , 0.01470694]))

(2, 4)


In [ ]:
# sparse matrices
# consider all subjects
# start by eeg
# channel neighbours

In [95]:
a,b = loo_(np.random.randn(48, 3465216))

# image

In [28]:
import os, sys, subprocess, wget, shutil


In [29]:
script = os.path.join(os.path.dirname(os.getcwd())+"/CalibrateNoise/", "main.py") 
file = "input/anat/original/"
clean = False

os.makedirs("img", exist_ok=True)
link_original = "https://s3.amazonaws.com/openneuro.org/ds004934/sub-SAXNES2s001/func/sub-SAXNES2s001_task-DOTS_run-001_bold.nii.gz?versionId=R0fwRS9fxw8CcPZnb4zYsw9I5v19aAbP"
wget.download(link_original)
os.makedirs(file, exist_ok=True)
file_ = [os.path.join(root, file) for root, _, files in os.walk(os.getcwd()) for file in files if file.endswith(".gz")]
shutil.copy2(file_[0], file)
os.remove(file_[0])
print(f"\nOriginal file downloaded.") 


100% [....................................................] 60545196 / 60545196
Original file downloaded.


In [31]:
result = subprocess.run(
    ["python3", script, file],
    capture_output=True,
    text=True
)
print(result.stdout)
if clean:
    ! rm -r input
    print(f"Cleaning up finished")  


Total time taken: 00:00:23



In [91]:
mne.set_log_level("ERROR")

BIDS_ROOT = "/staff/vincentajoubi/wp15-chrono-T/usecase-2.4/input"

vhdr_path, channels_tsv = None, None
for root, dirs, files in os.walk(BIDS_ROOT):
    for f in files:
        if f.endswith("_eeg.vhdr"):
            vhdr_path = os.path.join(root, f)
        if f.endswith("_channels.tsv"):
            channels_tsv = os.path.join(root, f)
    if vhdr_path and channels_tsv:
        break

raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)

channels_df = pd.read_csv(channels_tsv, sep="\t")
chan_types = {row["name"]: row["type"].lower() for _, row in channels_df.iterrows()}
raw.pick(channels_df["name"].tolist())
raw.set_channel_types(chan_types)

montage = mne.channels.make_standard_montage("standard_1020")
raw.set_montage(montage, match_case=False)

adjacency, ch_names = mne.channels.find_ch_adjacency(raw.info, ch_type="eeg")
dense = adjacency.toarray()

for idx, ch in enumerate(ch_names):
    print(np.stack([raw.get_data(picks=n) for n, flag in zip(ch_names, dense[idx]) if flag]).reshape(1, -1).shape)



(1, 2165760)
(1, 2598912)
(1, 2598912)
(1, 3032064)
(1, 2598912)
(1, 3032064)
(1, 3465216)
(1, 2165760)
(1, 2598912)
(1, 2598912)
(1, 2598912)
(1, 2165760)
(1, 2598912)
(1, 3032064)
(1, 3032064)
(1, 2165760)
(1, 2598912)
(1, 2598912)
(1, 2598912)
(1, 3032064)
(1, 3032064)
(1, 3032064)
(1, 2598912)
(1, 3032064)
(1, 3465216)
(1, 2165760)
(1, 2598912)
(1, 2598912)
(1, 2598912)
(1, 2165760)
